In [13]:
# Mechanics Roster Optimization
# Import required libraries
import pandas as pd
from ortools.linear_solver import pywraplp

# Load data files
print("Loading data files...")

# Load mechanic skills dataset
mechanic_skills_df = pd.read_excel('../data/mechanic_skills_dataset.xlsx')
print("\nMechanic Skills Dataset:")
print(mechanic_skills_df.head())
print(f"\nShape: {mechanic_skills_df.shape}")

# Extract mechanic IDs
mechanics = sorted(mechanic_skills_df['mechanic_id'].unique().tolist())
print(f"\nMechanics: {mechanics}")
print(f"Number of mechanics: {len(mechanics)}")

# Load base aircraft schedule
base_schedule_df = pd.read_excel('../data/base_aircraft_schedule.xlsx')
print("\n\nBase Aircraft Schedule:")
print(base_schedule_df.head())
print(f"\nShape: {base_schedule_df.shape}")

# Extract unique base IDs
bases = sorted(base_schedule_df['base_id'].unique().tolist())
print(f"\nBases: {bases}")
print(f"Number of bases: {len(bases)}")

# Extract periods (groups)
periods = sorted(base_schedule_df['period'].unique().tolist())
print(f"\nPeriods (Groups): {periods}")

# Extract shifts
shifts = sorted(base_schedule_df['shift'].unique().tolist())
print(f"Shifts: {shifts} (1=day, 2=night)")

# Load cost matrix
cost_matrix_df = pd.read_excel('../data/cost_matrix.xlsx')
print("\n\nCost Matrix:")
print(cost_matrix_df.head())
print(f"\nShape: {cost_matrix_df.shape}")

# Map cost columns A, B, C to base IDs
# Assuming A→1, B→2, C→3 (verify from base_schedule)
base_column_mapping = {'A': 1, 'B': 2, 'C': 3}
print(f"\nBase column mapping: {base_column_mapping}")

# Create cost dictionary: cost[mechanic_id, base_id]
cost_dict = {}
for _, row in cost_matrix_df.iterrows():
    mechanic_id = int(row['id'])
    for col, base_id in base_column_mapping.items():
        if col in cost_matrix_df.columns:
            cost_dict[(mechanic_id, base_id)] = float(row[col])

print(f"\nCost dictionary created with {len(cost_dict)} entries")
print("Sample costs:")
for i, (key, val) in enumerate(list(cost_dict.items())[:5]):
    print(f"  Mechanic {key[0]} to Base {key[1]}: {val}")

# Verify data consistency
print("\n\nData Validation:")
print(f"Mechanics in skills dataset: {len(mechanics)}")
print(f"Mechanics in cost matrix: {len(cost_matrix_df)}")
print(f"Bases in schedule: {bases}")
print(f"Bases in cost matrix: {list(base_column_mapping.values())}")

# Check for inspector skill columns
inspector_skill_columns = [col for col in mechanic_skills_df.columns if col.endswith('_inspec')]
print(f"\nInspector skill columns found: {len(inspector_skill_columns)}")
if inspector_skill_columns:
    print(f"  Inspector columns: {inspector_skill_columns[:5]}..." if len(inspector_skill_columns) > 5 else f"  Inspector columns: {inspector_skill_columns}")

# Check for inspector requirements in base schedule
inspector_req_columns = [col for col in base_schedule_df.columns if col.endswith('_inspec')]
print(f"Inspector requirement columns in base schedule: {len(inspector_req_columns)}")
if inspector_req_columns:
    print(f"  Inspector requirement columns: {inspector_req_columns[:5]}..." if len(inspector_req_columns) > 5 else f"  Inspector requirement columns: {inspector_req_columns}")

Loading data files...

Mechanic Skills Dataset:
   mechanic_id  aw139_af  aw139_r  aw139_av  h175_af  h175_r  h175_av  \
0            1         1        1         0        0       0        0   
1            2         0        0         1        0       0        1   
2            3         0        0         1        1       1        1   
3            4         1        1         0        0       0        1   
4            5         0        0         1        0       0        0   

   sk92_af  sk92_r  sk92_av  aw139_af_inspec  aw139_r_inspec  aw139_av_inspec  \
0        1       1        0                1               1                0   
1        0       0        1                0               0                0   
2        0       0        0                0               0                0   
3        0       0        0                0               1                0   
4        1       1        1                0               0                1   

   h175_af_inspec  h175_r_

In [14]:
# Set up OR-Tools solver
print("Setting up OR-Tools solver...")

# Create solver instance (using SCIP for mixed integer programming)
solver = pywraplp.Solver.CreateSolver('SCIP')

if not solver:
    print("SCIP solver not available, using CBC instead")
    solver = pywraplp.Solver.CreateSolver('CBC')

print(f"Solver: {solver.SolverVersion()}")

# Create decision variables: x[mechanic, base, group, shift]
# x[m, b, g, s] = 1 if mechanic m is assigned to base b in group g with shift s
# x[m, b, g, s] = 0 otherwise

print("\nCreating decision variables...")
x = {}

for m in mechanics:
    for b in bases:
        for g in periods:  # periods are the groups
            for s in shifts:
                var_name = f'x_m{m}_b{b}_g{g}_s{s}'
                x[(m, b, g, s)] = solver.IntVar(0, 1, var_name)

print(f"Created {len(x)} binary decision variables")
print(f"Sample variable: {x[(mechanics[0], bases[0], periods[0], shifts[0])]}")

Setting up OR-Tools solver...
Solver: SCIP 10.0.0 [LP solver: SoPlex 8.0.0]

Creating decision variables...
Created 588 binary decision variables
Sample variable: x_m1_b1_g1_s1


In [15]:
# Constraint 1: Each mechanic can work in at most one shift, one group, and one base
# Σ(b∈Bases) Σ(g∈Groups) Σ(s∈Shifts) x[m, b, g, s] ≤ 1  ∀m ∈ Mechanics

print("Adding first constraint: Each mechanic ≤ 1 assignment...")

constraint_count = 0
for m in mechanics:
    constraint = solver.Constraint(0, 1, f'mechanic_{m}_single_assignment')
    for b in bases:
        for g in periods:
            for s in shifts:
                constraint.SetCoefficient(x[(m, b, g, s)], 1)
    constraint_count += 1

print(f"Added {constraint_count} constraints (one per mechanic)")
print(f"Total constraints so far: {solver.NumConstraints()}")

Adding first constraint: Each mechanic ≤ 1 assignment...
Added 49 constraints (one per mechanic)
Total constraints so far: 49


In [16]:
# Constraint 2: Skill Coverage Constraint
# For each (base, period, shift) combination, the pool of assigned mechanics must 
# collectively have all required skills for all aircraft types present at that base
# For each aircraft type a present at (b, p, s):
#   Σ(m ∈ Mechanics with skill a_af) x[m, b, p, s] ≥ 1
#   Σ(m ∈ Mechanics with skill a_r) x[m, b, p, s] ≥ 1
#   Σ(m ∈ Mechanics with skill a_av) x[m, b, p, s] ≥ 1

print("Adding skill coverage constraints...")

# Aircraft types and their corresponding skill columns
aircraft_types = ['aw139', 'h175', 'sk92']
skill_types = ['_af', '_r', '_av']

# Create a mapping of mechanics to their skills for quick lookup
mechanic_skills = {}
mechanic_inspector_skills = {}  # For inspector skills
for _, row in mechanic_skills_df.iterrows():
    m = int(row['mechanic_id'])
    mechanic_skills[m] = {}
    mechanic_inspector_skills[m] = {}
    for aircraft in aircraft_types:
        for skill in skill_types:
            col_name = f"{aircraft}{skill}"
            if col_name in mechanic_skills_df.columns:
                mechanic_skills[m][col_name] = int(row[col_name])
            # Also check for inspector skills
            inspector_col_name = f"{aircraft}{skill}_inspec"
            if inspector_col_name in mechanic_skills_df.columns:
                mechanic_inspector_skills[m][inspector_col_name] = int(row[inspector_col_name])

constraint_count = 0

# Iterate through each row in base_aircraft_schedule
for _, row in base_schedule_df.iterrows():
    base_id = int(row['base_id'])
    period = int(row['period'])
    shift = int(row['shift'])
    
    # Check which aircraft types are present (count > 0)
    for aircraft in aircraft_types:
        if aircraft in base_schedule_df.columns:
            aircraft_count = row[aircraft]
            if aircraft_count > 0:  # Aircraft type is present
                # For each skill type (_af, _r, _av), ensure at least one mechanic with that skill is assigned
                for skill in skill_types:
                    skill_name = f"{aircraft}{skill}"
                    constraint = solver.Constraint(1, solver.infinity(), 
                                                  f'skill_{skill_name}_base{base_id}_period{period}_shift{shift}')
                    
                    # Find all mechanics with this skill
                    for m in mechanics:
                        if mechanic_skills[m].get(skill_name, 0) == 1:
                            constraint.SetCoefficient(x[(m, base_id, period, shift)], 1)
                    
                    constraint_count += 1

print(f"Added {constraint_count} skill coverage constraints")
print(f"Total constraints: {solver.NumConstraints()}")

Adding skill coverage constraints...
Added 66 skill coverage constraints
Total constraints: 115


In [17]:
# Constraint 3: Inspector Coverage Constraint
# For each (base, period, shift) combination, if an inspector skill is required,
# the pool of assigned mechanics must include at least one mechanic with that inspector skill
# For each inspector requirement column in base_schedule_df:
#   If value > 0 at (b, p, s), then:
#     Σ(m ∈ Mechanics with inspector skill) x[m, b, p, s] ≥ 1

print("Adding inspector coverage constraints...")

inspector_constraint_count = 0

# Get all inspector requirement columns from base schedule
inspector_req_columns = [col for col in base_schedule_df.columns if col.endswith('_inspec')]

if inspector_req_columns:
    # Iterate through each row in base_aircraft_schedule
    for _, row in base_schedule_df.iterrows():
        base_id = int(row['base_id'])
        period = int(row['period'])
        shift = int(row['shift'])
        
        # Check each inspector requirement column
        for inspector_col in inspector_req_columns:
            if inspector_col in base_schedule_df.columns:
                inspector_required = row[inspector_col]
                # Check if inspector is required (value > 0)
                if pd.notna(inspector_required) and inspector_required > 0:
                    # Create constraint: at least one mechanic with this inspector skill must be assigned
                    constraint = solver.Constraint(1, solver.infinity(), 
                                                  f'inspector_{inspector_col}_base{base_id}_period{period}_shift{shift}')
                    
                    # Find all mechanics with this inspector skill
                    for m in mechanics:
                        if mechanic_inspector_skills[m].get(inspector_col, 0) == 1:
                            constraint.SetCoefficient(x[(m, base_id, period, shift)], 1)
                    
                    inspector_constraint_count += 1
    
    print(f"Added {inspector_constraint_count} inspector coverage constraints")
    print(f"Total constraints: {solver.NumConstraints()}")
else:
    print("No inspector requirement columns found in base schedule. Skipping inspector constraints.")

Adding inspector coverage constraints...
Added 19 inspector coverage constraints
Total constraints: 134


In [18]:
# Constraint 4: No Self-Inspection Constraint
# If a mechanic is assigned as an inspector for a skill, they cannot be the only one
# doing that work. At least one OTHER mechanic with the regular skill must also be assigned.
# This prevents a mechanic from inspecting their own work.
# Mathematical formulation:
# For each (base, period, shift) and inspector requirement:
#   If mechanic m_inspector is assigned AND has inspector skill, then:
#     Σ(m ∈ other_mechanics with regular skill) x[m, b, g, s] ≥ 1
# Which can be written as:
#   x[m_inspector, b, g, s] ≤ Σ(m ∈ other_mechanics) x[m, b, g, s]

print("Adding no self-inspection constraints...")

no_self_inspection_count = 0

# Get all inspector requirement columns from base schedule
inspector_req_columns = [col for col in base_schedule_df.columns if col.endswith('_inspec')]

if inspector_req_columns:
    # Iterate through each row in base_aircraft_schedule
    for _, row in base_schedule_df.iterrows():
        base_id = int(row['base_id'])
        period = int(row['period'])
        shift = int(row['shift'])
        
        # Check each inspector requirement column
        for inspector_col in inspector_req_columns:
            if inspector_col in base_schedule_df.columns:
                inspector_required = row[inspector_col]
                # Check if inspector is required (value > 0)
                if pd.notna(inspector_required) and inspector_required > 0:
                    # Extract the regular skill name from inspector column
                    # e.g., "aw139_af_inspec" -> "aw139_af"
                    regular_skill_name = inspector_col.replace('_inspec', '')
                    
                    # For each mechanic who could be the inspector (has inspector skill)
                    for m_inspector in mechanics:
                        if mechanic_inspector_skills[m_inspector].get(inspector_col, 0) == 1:
                            # This mechanic could be assigned as inspector
                            # We need to ensure that if they are assigned, at least one
                            # OTHER mechanic with the regular skill is also assigned
                            
                            # Count how many OTHER mechanics (not m_inspector) have the regular skill
                            other_mechanics_with_skill = [
                                m for m in mechanics 
                                if m != m_inspector 
                                and mechanic_skills[m].get(regular_skill_name, 0) == 1
                            ]
                            
                            if other_mechanics_with_skill:
                                # Create constraint: If m_inspector is assigned, then at least one
                                # other mechanic with the regular skill must also be assigned
                                # x[m_inspector, b, g, s] ≤ Σ(m ∈ other_mechanics) x[m, b, g, s]
                                
                                constraint = solver.Constraint(
                                    -solver.infinity(), 0,
                                    f'no_self_inspect_{inspector_col}_base{base_id}_period{period}_shift{shift}_inspector{m_inspector}'
                                )
                                
                                # Left side: x[m_inspector, b, g, s]
                                constraint.SetCoefficient(x[(m_inspector, base_id, period, shift)], 1)
                                
                                # Right side: -Σ(m ∈ other_mechanics) x[m, b, g, s]
                                for m_other in other_mechanics_with_skill:
                                    constraint.SetCoefficient(x[(m_other, base_id, period, shift)], -1)
                                
                                no_self_inspection_count += 1
    
    print(f"Added {no_self_inspection_count} no self-inspection constraints")
    print(f"Total constraints: {solver.NumConstraints()}")
else:
    print("No inspector requirements found. Skipping no self-inspection constraints.")

Adding no self-inspection constraints...
Added 215 no self-inspection constraints
Total constraints: 349


Creating avoidance penalty variables and constraints...
Creating variables for 35 unique avoidance pairs...
Created 420 avoidance penalty variables
Added 1260 linearization constraints for avoidance penalties
Total constraints: 1609
Total variables: 1008


In [19]:
# Objective Function: Minimize total cost of moving mechanics to bases
# Minimize: Σ(m∈Mechanics) Σ(b∈Bases) Σ(g∈Groups) Σ(s∈Shifts) cost[m, b] * x[m, b, g, s]

print("Creating objective function...")

objective = solver.Objective()

for m in mechanics:
    for b in bases:
        # Get cost for this mechanic-base pair
        cost = cost_dict.get((m, b), 0)
        for g in periods:
            for s in shifts:
                objective.SetCoefficient(x[(m, b, g, s)], cost)

objective.SetMinimization()

print("Objective function created: Minimize total movement cost")
print(f"Number of variables in objective: {len(x)}")
print(f"Model summary:")
print(f"  Variables: {solver.NumVariables()}")
print(f"  Constraints: {solver.NumConstraints()}")

Creating objective function...
Objective function created: Minimize total movement cost
Number of variables in objective: 588
Model summary:
  Variables: 588
  Constraints: 349


In [20]:
# Solve the optimization problem
print("Solving the optimization problem...")
print("This may take a few moments...\n")

status = solver.Solve()

# Check solution status
if status == pywraplp.Solver.OPTIMAL:
    print("✓ Optimal solution found!\n")
    print(f"Optimal total cost: {objective.Value():.2f}\n")
    
    # Display assignments
    print("Mechanic Assignments:")
    print("-" * 80)
    print(f"{'Mechanic':<10} {'Base':<8} {'Group':<8} {'Shift':<8} {'Cost':<10}")
    print("-" * 80)
    
    assignments = []
    total_cost = 0
    for m in mechanics:
        for b in bases:
            for g in periods:
                for s in shifts:
                    if x[(m, b, g, s)].solution_value() > 0.5:  # Binary variable is 1
                        cost = cost_dict.get((m, b), 0)
                        total_cost += cost
                        shift_name = "Day" if s == 1 else "Night"
                        print(f"{m:<10} {b:<8} {g:<8} {shift_name:<8} {cost:<10.2f}")
                        assignments.append({
                            'mechanic_id': m,
                            'base_id': b,
                            'group': g,
                            'shift': s,
                            'shift_name': shift_name,
                            'cost': cost
                        })
    
    print("-" * 80)
    print(f"\nTotal assignments: {len(assignments)}")
    print(f"Total cost: {total_cost:.2f}")
    print(f"\nUnassigned mechanics: {len(mechanics) - len(assignments)}")
    
elif status == pywraplp.Solver.FEASIBLE:
    print("✓ Feasible solution found (may not be optimal)\n")
    print(f"Total cost: {objective.Value():.2f}\n")
elif status == pywraplp.Solver.INFEASIBLE:
    print("✗ Problem is infeasible - no solution exists that satisfies all constraints")
elif status == pywraplp.Solver.UNBOUNDED:
    print("✗ Problem is unbounded")
else:
    print(f"✗ Solver status: {status}")

Solving the optimization problem...
This may take a few moments...

✓ Optimal solution found!

Optimal total cost: 648.00

Mechanic Assignments:
--------------------------------------------------------------------------------
Mechanic   Base     Group    Shift    Cost      
--------------------------------------------------------------------------------
1          2        2        Night    14.00     
2          3        2        Night    10.00     
3          1        1        Day      27.00     
4          2        2        Night    0.00      
5          2        1        Night    25.00     
7          1        2        Night    42.00     
8          3        2        Day      35.00     
9          2        2        Day      11.00     
10         3        2        Day      54.00     
13         1        2        Day      41.00     
17         1        1        Night    1.00      
18         3        1        Night    39.00     
19         1        1        Night    18.00     
21     

In [ ]:
# Generate Excel output in calendar format
print("\n" + "="*80)
print("GENERATING EXCEL OUTPUT")
print("="*80)

if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    from openpyxl import Workbook
    from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
    from openpyxl.utils import get_column_letter
    
    # Create workbook
    wb = Workbook()
    ws = wb.active
    ws.title = "Roster"
    
    # Define styles
    header_fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")
    header_font = Font(bold=True, color="FFFFFF", size=11)
    section_fill = PatternFill(start_color="D9E1F2", end_color="D9E1F2", fill_type="solid")
    section_font = Font(bold=True, size=11)
    center_align = Alignment(horizontal="center", vertical="center")
    thin_border = Border(
        left=Side(style='thin'),
        right=Side(style='thin'),
        top=Side(style='thin'),
        bottom=Side(style='thin')
    )
    
    # Column headers (matching sample format - no Shift column header)
    headers = ['Base', 'Roster', 'Airframe', 'Engine', 'Avionics', 'Position'] + [f'Day {i}' for i in range(1, 31)]
    ws.append(headers)
    
    # Style header row
    for col_idx, header in enumerate(headers, 1):
        cell = ws.cell(row=1, column=col_idx)
        cell.fill = header_fill
        cell.font = header_font
        cell.alignment = center_align
        cell.border = thin_border
    
    # Base mapping (A=1, B=2, C=3)
    base_letter_map = {1: 'A', 2: 'B', 3: 'C'}
    
    # Helper function to determine position
    def get_position(mechanic_id, has_inspector_skill_for_shift, mechanic_skills_dict):
        # Inspector override: if selected as inspector for this shift, always Inspector
        if has_inspector_skill_for_shift:
            return 'Inspector'
        
        # Check skills across all aircraft models
        has_airframe = False
        has_engine = False
        has_avionics = False
        
        for aircraft in ['aw139', 'h175', 'sk92']:
            if mechanic_skills_dict.get(f'{aircraft}_af', 0) == 1:
                has_airframe = True
            if mechanic_skills_dict.get(f'{aircraft}_r', 0) == 1:
                has_engine = True
            if mechanic_skills_dict.get(f'{aircraft}_av', 0) == 1:
                has_avionics = True
        
        # Position rules:
        # - If mechanic has avionics skill and any of airframe/engine missing → Avionic
        # - If mechanic has airframe and/or engine and no avionics → Mechanic
        # - If mechanic has all three skills → Mechanic
        if has_avionics and not (has_airframe and has_engine):
            return 'Avionic'
        if has_airframe or has_engine or has_avionics:
            return 'Mechanic'
        # If no skills flagged, default to Mechanic
        return 'Mechanic'
    
    # Helper function to check if mechanic has skill type across all aircraft
    def has_skill_type(mechanic_id, skill_type, mechanic_skills_dict):
        for aircraft in ['aw139', 'h175', 'sk92']:
            if mechanic_skills_dict.get(f'{aircraft}{skill_type}', 0) == 1:
                return True
        return False
    
    # Organize assignments by base and shift
    assignments_by_base_shift = {}
    for assignment in assignments:
        base_id = assignment['base_id']
        shift = assignment['shift']
        group = assignment['group']
        key = (base_letter_map.get(base_id, base_id), shift)
        
        if key not in assignments_by_base_shift:
            assignments_by_base_shift[key] = []
        assignments_by_base_shift[key].append({
            'mechanic_id': assignment['mechanic_id'],
            'group': group,
            'shift': shift
        })
    
    # Sort by base (A, B, C) and shift (1=Day, 2=Night)
    sorted_keys = sorted(assignments_by_base_shift.keys(), key=lambda x: (x[0], x[1]))
    
    current_row = 2
    
    # Process each base and shift combination
    for (base_letter, shift_num) in sorted_keys:
        shift_name = "Day Shift" if shift_num == 1 else "Night Shift"
        
        # Add section header (matching sample format)
        # In sample, section header only has "Day Shift" or "Night Shift" in first column
        ws.append([shift_name] + [None] * (len(headers) - 1))
        section_row = current_row
        # Style only the first column for section header (matching sample)
        cell = ws.cell(row=section_row, column=1)
        cell.fill = section_fill
        cell.font = section_font
        cell.alignment = Alignment(horizontal="left", vertical="center")
        cell.border = thin_border
        # Style remaining columns with borders but no fill
        for col_idx in range(2, len(headers) + 1):
            cell = ws.cell(row=section_row, column=col_idx)
            cell.border = thin_border
        current_row += 1
        
        # Get mechanics for this base and shift
        base_assignments = assignments_by_base_shift[(base_letter, shift_num)]
        
        # Sort mechanics by ID
        base_assignments.sort(key=lambda x: x['mechanic_id'])
        
        # For each mechanic
        for assignment_info in base_assignments:
            mechanic_id = assignment_info['mechanic_id']
            group = assignment_info['group']
            
            # Get mechanic skills
            m_skills = mechanic_skills.get(mechanic_id, {})
            
            # Check if mechanic is inspector for this shift
            # Check if mechanic has any inspector skills that are required at this base/shift
            is_inspector = False
            if inspector_req_columns:
                for _, row in base_schedule_df.iterrows():
                    if int(row['base_id']) == [k for k, v in base_letter_map.items() if v == base_letter][0] and \
                       int(row['shift']) == shift_num:
                        for inspector_col in inspector_req_columns:
                            if inspector_col in base_schedule_df.columns:
                                inspector_required = row[inspector_col]
                                if pd.notna(inspector_required) and inspector_required > 0:
                                    if mechanic_inspector_skills.get(mechanic_id, {}).get(inspector_col, 0) == 1:
                                        # Check if this mechanic is actually assigned and serving as inspector
                                        # (i.e., there are other mechanics with the regular skill)
                                        regular_skill_name = inspector_col.replace('_inspec', '')
                                        other_mechanics = [
                                            a['mechanic_id'] for a in base_assignments
                                            if a['mechanic_id'] != mechanic_id
                                            and mechanic_skills.get(a['mechanic_id'], {}).get(regular_skill_name, 0) == 1
                                        ]
                                        if other_mechanics:
                                            is_inspector = True
                                            break
                        if is_inspector:
                            break
            
            # Determine position
            position = get_position(mechanic_id, is_inspector, m_skills)
            
            # Check skill boxes
            has_airframe = has_skill_type(mechanic_id, '_af', m_skills)
            has_engine = has_skill_type(mechanic_id, '_r', m_skills)
            has_avionics = has_skill_type(mechanic_id, '_av', m_skills)
            
            # Create row data (matching sample format)
            # Note: Sample has an unlabeled column with True/False for shift, but we'll skip it
            # to match the exact column structure
            # Use checkbox-style symbols: '☑' for yes/working, '☐' for no/not working
            row_data = [
                base_letter,
                f'Mechanic {mechanic_id}',
                '☑' if has_airframe else '☐',
                '☑' if has_engine else '☐',
                '☑' if has_avionics else '☐',
                position
            ]
            
            # Add day columns (1-30) with checkbox-style symbols
            # Days 1-15: Group 1, Days 16-30: Group 2
            for day in range(1, 31):
                if (day <= 15 and group == 1) or (day > 15 and group == 2):
                    row_data.append('☑')  # Working
                else:
                    row_data.append('☐')  # Not working
            
            ws.append(row_data)
            
            # Style the row
            for col_idx in range(1, len(headers) + 1):
                cell = ws.cell(row=current_row, column=col_idx)
                cell.alignment = center_align
                cell.border = thin_border
            
            current_row += 1
    
    # Auto-adjust column widths
    for col_idx, header in enumerate(headers, 1):
        if col_idx <= 6:  # Base through Position
            ws.column_dimensions[get_column_letter(col_idx)].width = 12
        else:  # Day columns
            ws.column_dimensions[get_column_letter(col_idx)].width = 8
    
    # Save file
    output_filename = '../data/roster_output.xlsx'
    wb.save(output_filename)
    print(f"✓ Excel file created: {output_filename}")
    print(f"✓ Total rows: {current_row - 1}")
    print(f"✓ Format: Calendar view with Day 1-30 columns")
    
else:
    print("⚠ No solution available. Cannot generate Excel output.")


GENERATING EXCEL OUTPUT
✓ Excel file created: ../data/roster_output.xlsx
✓ Total rows: 38
✓ Format: Calendar view with Day 1-30 columns


In [22]:
# After solving, add this verification cell
print("\n" + "="*80)
print("OPTIMALITY VERIFICATION")
print("="*80)

if status == pywraplp.Solver.OPTIMAL:
    print("✓ Solver Status: OPTIMAL")
    print(f"✓ Objective Value: {objective.Value():.2f}")
    
    # Check if solver provides best bound (lower bound for minimization)
    try:
        best_bound = solver.Objective().BestBound()
        gap = abs(objective.Value() - best_bound)
        gap_percent = (gap / objective.Value()) * 100 if objective.Value() != 0 else 0
        print(f"✓ Best Bound (Lower Bound): {best_bound:.2f}")
        print(f"✓ Optimality Gap: {gap:.6f} ({gap_percent:.6f}%)")
        if gap < 1e-6:
            print("✓ Gap is essentially zero - solution is proven optimal!")
        else:
            print(f"⚠ Gap exists but within tolerance")
    except:
        print("ℹ Best bound not available from this solver")
    
    # Solver statistics
    print(f"\nSolver Statistics:")
    print(f"  Wall time: {solver.wall_time():.2f} ms")
    print(f"  Iterations: {solver.iterations()}")
    print(f"  Nodes: {solver.nodes()}")
    
else:
    print(f"✗ Solver Status: {status}")
    if status == pywraplp.Solver.FEASIBLE:
        print("⚠ Solution is feasible but may not be optimal")
        print("  Consider increasing time limit or checking solver parameters")


OPTIMALITY VERIFICATION
✓ Solver Status: OPTIMAL
✓ Objective Value: 648.00
✓ Best Bound (Lower Bound): 648.00
✓ Optimality Gap: 0.000000 (0.000000%)
✓ Gap is essentially zero - solution is proven optimal!

Solver Statistics:
  Wall time: 44198.00 ms
  Iterations: 48393
  Nodes: 988


In [23]:
# Verify constraints are satisfied
print("\n" + "="*80)
print("CONSTRAINT VERIFICATION")
print("="*80)

# Verify constraint 1: Each mechanic ≤ 1 assignment
violations = []
for m in mechanics:
    total_assignments = sum(x[(m, b, g, s)].solution_value() 
                          for b in bases for g in periods for s in shifts)
    if total_assignments > 1.001:  # Small tolerance for floating point
        violations.append(f"Mechanic {m} has {total_assignments} assignments")
    elif total_assignments < -0.001:
        violations.append(f"Mechanic {m} has negative assignment: {total_assignments}")

if not violations:
    print("✓ Constraint 1: All mechanics have ≤ 1 assignment")
else:
    print("✗ Constraint 1 violations:")
    for v in violations:
        print(f"  {v}")

# Verify skill coverage constraints
skill_violations = []
for _, row in base_schedule_df.iterrows():
    base_id = int(row['base_id'])
    period = int(row['period'])
    shift = int(row['shift'])
    
    for aircraft in ['aw139', 'h175', 'sk92']:
        if aircraft in base_schedule_df.columns and row[aircraft] > 0:
            for skill in ['_af', '_r', '_av']:
                skill_name = f"{aircraft}{skill}"
                assigned_with_skill = sum(
                    x[(m, base_id, period, shift)].solution_value()
                    for m in mechanics
                    if mechanic_skills[m].get(skill_name, 0) == 1
                )
                if assigned_with_skill < 0.999:  # Should be ≥ 1
                    skill_violations.append(
                        f"Base {base_id}, Period {period}, Shift {shift}: "
                        f"Missing {skill_name} (found {assigned_with_skill})"
                    )

if not skill_violations:
    print("✓ Constraint 2: All skill coverage requirements met")
else:
    print("✗ Skill coverage violations:")
    for v in skill_violations[:10]:  # Show first 10
        print(f"  {v}")
    if len(skill_violations) > 10:
        print(f"  ... and {len(skill_violations) - 10} more")

# Verify inspector coverage constraints
inspector_violations = []
inspector_req_columns = [col for col in base_schedule_df.columns if col.endswith('_inspec')]

if inspector_req_columns:
    for _, row in base_schedule_df.iterrows():
        base_id = int(row['base_id'])
        period = int(row['period'])
        shift = int(row['shift'])
        
        for inspector_col in inspector_req_columns:
            if inspector_col in base_schedule_df.columns:
                inspector_required = row[inspector_col]
                if pd.notna(inspector_required) and inspector_required > 0:
                    # Check if at least one mechanic with this inspector skill is assigned
                    assigned_with_inspector_skill = sum(
                        x[(m, base_id, period, shift)].solution_value()
                        for m in mechanics
                        if mechanic_inspector_skills[m].get(inspector_col, 0) == 1
                    )
                    if assigned_with_inspector_skill < 0.999:  # Should be ≥ 1
                        inspector_violations.append(
                            f"Base {base_id}, Period {period}, Shift {shift}: "
                            f"Missing inspector {inspector_col} (found {assigned_with_inspector_skill})"
                        )
    
    if not inspector_violations:
        print("✓ Constraint 3: All inspector coverage requirements met")
    else:
        print("✗ Inspector coverage violations:")
        for v in inspector_violations[:10]:  # Show first 10
            print(f"  {v}")
        if len(inspector_violations) > 10:
            print(f"  ... and {len(inspector_violations) - 10} more")
else:
    print("ℹ Constraint 3: No inspector requirements found (skipped)")

# Verify no self-inspection constraints
self_inspection_violations = []

if inspector_req_columns:
    for _, row in base_schedule_df.iterrows():
        base_id = int(row['base_id'])
        period = int(row['period'])
        shift = int(row['shift'])
        
        for inspector_col in inspector_req_columns:
            if inspector_col in base_schedule_df.columns:
                inspector_required = row[inspector_col]
                if pd.notna(inspector_required) and inspector_required > 0:
                    # Extract the regular skill name
                    regular_skill_name = inspector_col.replace('_inspec', '')
                    
                    # Check each mechanic who could be inspector
                    for m_inspector in mechanics:
                        if mechanic_inspector_skills[m_inspector].get(inspector_col, 0) == 1:
                            # If this mechanic is assigned
                            if x[(m_inspector, base_id, period, shift)].solution_value() > 0.5:
                                # Check if there's at least one OTHER mechanic with the regular skill assigned
                                other_mechanics_with_skill_assigned = sum(
                                    x[(m, base_id, period, shift)].solution_value()
                                    for m in mechanics
                                    if m != m_inspector
                                    and mechanic_skills[m].get(regular_skill_name, 0) == 1
                                )
                                
                                if other_mechanics_with_skill_assigned < 0.999:
                                    self_inspection_violations.append(
                                        f"Base {base_id}, Period {period}, Shift {shift}: "
                                        f"Mechanic {m_inspector} is inspector for {inspector_col} but no other "
                                        f"mechanic with {regular_skill_name} is assigned (self-inspection violation)"
                                    )
    
    if not self_inspection_violations:
        print("✓ Constraint 4: No self-inspection violations (inspectors have other mechanics to inspect)")
    else:
        print("✗ Self-inspection violations:")
        for v in self_inspection_violations[:10]:  # Show first 10
            print(f"  {v}")
        if len(self_inspection_violations) > 10:
            print(f"  ... and {len(self_inspection_violations) - 10} more")
else:
    print("ℹ Constraint 4: No inspector requirements found (skipped)")

print(f"\n✓ Total constraint violations: {len(violations) + len(skill_violations) + len(inspector_violations) + len(self_inspection_violations)}")


CONSTRAINT VERIFICATION
✓ Constraint 1: All mechanics have ≤ 1 assignment
✓ Constraint 2: All skill coverage requirements met
✓ Constraint 3: All inspector coverage requirements met
✓ Constraint 4: No self-inspection violations (inspectors have other mechanics to inspect)

✓ Total constraint violations: 0
